# Access InfluxDB from Python
Inspired by https://www.influxdata.com/blog/getting-started-python-influxdb/

In [13]:
from influxdb import InfluxDBClient
from ruamel.yaml import YAML
import pandas as pd
import datetime as dt

## Make the client and connect to a database

In [14]:
with open("secrets.yaml", "r") as file:
    secrets = YAML().load(file)

client = InfluxDBClient(host=secrets["influx"]["host"], 
                        port=secrets["influx"]["port"], 
                        username=secrets["influx"]["username"], 
                        password=secrets["influx"]["password"])

client.get_list_database()

[{'name': '_internal'},
 {'name': 'influx_db_1'},
 {'name': 'influx_db_2'},
 {'name': 'influx_db_3'},
 {'name': 'influx_db_4'}]

In [15]:
client.switch_database('influx_db_4')

## show series

In [38]:
points = [p["key"].replace("\\", "") for p in client.query("show series").get_points()]
for i, point in enumerate(points):
    print(i, point)

0 %,domain=sensor,entity_id=benni_battery
1 %,domain=sensor,entity_id=benni_moisture
2 %,domain=sensor,entity_id=cooper_se_charging_target
3 %,domain=sensor,entity_id=cooper_se_remaining_battery_percent
4 %,domain=sensor,entity_id=disk_use_percent
5 %,domain=sensor,entity_id=disk_use_percent_config
6 %,domain=sensor,entity_id=ewelink_ds01_battery
7 %,domain=sensor,entity_id=hp_laserjet_pro_m118_m119_black_cartridge_hp_cf294a
8 %,domain=sensor,entity_id=hp_laserjet_pro_m118_m119_imaging_drum_hp_cf232a
9 %,domain=sensor,entity_id=memory_use_percent
10 %,domain=sensor,entity_id=moto_g54_5g_battery_level
11 %,domain=sensor,entity_id=my_sensor
12 %,domain=sensor,entity_id=mz_shutter_ff_bath_illuminance
13 %,domain=sensor,entity_id=mz_shutter_ff_bath_illuminance_9
14 %,domain=sensor,entity_id=mz_shutter_ff_lara_l_illuminance
15 %,domain=sensor,entity_id=mz_shutter_ff_lara_m_illuminance
16 %,domain=sensor,entity_id=mz_shutter_ff_lara_r_illuminance
17 %,domain=sensor,entity_id=mz_shutter_ff_sl

## Query simple 

In [17]:
qstr = f'SELECT * FROM "m³/h" WHERE TIME >= now() -1d'
print(f"{qstr=}")
df = pd.DataFrame.from_records(client.query(qstr).get_points())
df.tail(3)

qstr='SELECT * FROM "m³/h" WHERE TIME >= now() -1d'


,time,domain,entity_id,mean_value
28,2024-06-01T18:00:00Z,sensor,watermeter_rate_per_time_unit,0.0075
29,2024-06-01T19:00:00Z,sensor,gasmeter2_rate_per_time_unit,0.0090
30,2024-06-01T19:00:00Z,sensor,watermeter_rate_per_time_unit,0.0210


In [18]:
df.time = pd.to_datetime(df.time)
df = df.set_index("time")
df.head()

,domain,entity_id,mean_value
time,,,
2024-05-31 21:00:00+00:00,sensor,gasmeter2_rate_per_time_unit,0.0870
2024-05-31 22:00:00+00:00,sensor,watermeter_rate_per_time_unit,0.0096
2024-06-01 03:00:00+00:00,sensor,watermeter_rate_per_time_unit,0.0222
2024-06-01 04:00:00+00:00,sensor,gasmeter2_rate_per_time_unit,0.0540
2024-06-01 05:00:00+00:00,sensor,gasmeter2_rate_per_time_unit,0.1272


In [19]:
df.entity_id.value_counts()

entity_id
gasmeter2_rate_per_time_unit     16
watermeter_rate_per_time_unit    15
Name: count, dtype: int64

## Query with datetime filter

In [20]:
unit = "m³"
entity_id = "watermeter_value"
selection = "mean_value"
qstr = f'SELECT {selection} FROM "{unit}" WHERE {entity_id=}'
qstr

'SELECT mean_value FROM "m³" WHERE entity_id=\'watermeter_value\''

Simple query string w/o datetime filter provides all data:

In [21]:
pd.DataFrame.from_records(client.query(qstr).get_points())

,time,mean_value
0,2023-10-29T14:00:00Z,579.340333
1,2023-10-29T15:00:00Z,579.342650
2,2023-10-29T16:00:00Z,579.360750
3,2023-10-29T17:00:00Z,579.378600
4,2023-10-29T18:00:00Z,579.287033
...,...,...
2880,2024-06-01T15:00:00Z,619.490500
2881,2024-06-01T16:00:00Z,619.496533
2882,2024-06-01T17:00:00Z,619.520500
2883,2024-06-01T18:00:00Z,619.545200


## Adding a datetime filter
InfluxDB is very intolerant about the quotes (`"`, `'`)

In [22]:
start_date = dt.datetime(2024,4,1).date()
start_time = dt.time(12, 0, 0)
rfc3339_format = '%Y-%m-%dT%H:%M:%S.00000000Z'
start_string = dt.datetime.combine(start_date, start_time).strftime(rfc3339_format)
start_string

'2024-04-01T12:00:00.00000000Z'

In [23]:
qstr = f"""SELECT {selection} FROM "{unit}" WHERE {entity_id=} AND time >= '{start_string}'"""
qstr

'SELECT mean_value FROM "m³" WHERE entity_id=\'watermeter_value\' AND time >= \'2024-04-01T12:00:00.00000000Z\''

In [24]:
pd.DataFrame.from_records(client.query(qstr).get_points()).set_index("time")

,mean_value
time,
2024-04-01T12:00:00Z,607.470350
2024-04-01T14:00:00Z,607.470350
2024-04-01T15:00:00Z,607.470350
2024-04-01T16:00:00Z,607.479475
2024-04-01T18:00:00Z,607.492200
...,...
2024-06-01T15:00:00Z,619.490500
2024-06-01T16:00:00Z,619.496533
2024-06-01T17:00:00Z,619.520500


# Database cleanup

## Deleting entity_id's with few data points

In [58]:
DANGER_FLAG_DELETE_IN_DATABASE = False  # set to True only if you know what you are doing

In [60]:
records = list()
points = [p["key"].replace("\\", "") for p in client.query("show series").get_points()]
for point in points:
    unit, _, entity_id = point.split(",")
    entity_id = entity_id.split("=")[-1]
    qstr = f"""SELECT * FROM "{unit}" WHERE entity_id = '{entity_id}'"""
    data = pd.DataFrame.from_records(client.query(qstr).get_points())
    if len(data) < 10:
        delete_str = f"""DELETE FROM "{unit}" WHERE entity_id = '{entity_id}'"""
        print(f"{delete_str=}, because {len(data)=}")
        if DANGER_FLAG_DELETE_IN_DATABASE:
            client.query(delete_str)
    else:    
        records.append({"unit": unit, "entity_id": entity_id, "length": len(data)})
df = pd.DataFrame.from_records(records)

delete_str='DELETE FROM "%" WHERE entity_id = \'disk_use_percent_config\'', because len(data)=1
delete_str='DELETE FROM "%" WHERE entity_id = \'ewelink_ds01_battery\'', because len(data)=1
delete_str='DELETE FROM "%" WHERE entity_id = \'hp_laserjet_pro_m118_m119_black_cartridge_hp_cf294a\'', because len(data)=7
delete_str='DELETE FROM "%" WHERE entity_id = \'hp_laserjet_pro_m118_m119_imaging_drum_hp_cf232a\'', because len(data)=7
delete_str='DELETE FROM "%" WHERE entity_id = \'mz_shutter_ff_bath_illuminance_9\'', because len(data)=1
delete_str='DELETE FROM "%" WHERE entity_id = \'plant_sensor_e3fd_battery\'', because len(data)=1
delete_str='DELETE FROM "%" WHERE entity_id = \'plant_sensor_e3fd_moisture\'', because len(data)=1
delete_str='DELETE FROM "%" WHERE entity_id = \'shellyht_18c268_battery\'', because len(data)=1
delete_str='DELETE FROM "%" WHERE entity_id = \'shellyht_18c268_humidity\'', because len(data)=1
delete_str='DELETE FROM "%" WHERE entity_id = \'shellyht_cc2983_battery

In [48]:
unit = "µS/cm"
entity_id = "plant_sensor_e3fd_conductivity"
qstr = f"""SELECT * FROM "{unit}" WHERE entity_id = '{entity_id}'"""
print(qstr)
pd.DataFrame.from_records(client.query(qstr).get_points())

SELECT * FROM "µS/cm" WHERE entity_id = 'plant_sensor_e3fd_conductivity'


""


In [47]:
qstr = f"""DELETE FROM "{unit}" WHERE entity_id = '{entity_id}'"""
print(qstr)
client.query(qstr)

DELETE FROM "µS/cm" WHERE entity_id = 'plant_sensor_e3fd_conductivity'


ResultSet({})

In [42]:
pd.options.display.max_rows = 300
df.sort_values("length")

,unit,entity_id,length
293,µS/cm,plant_sensor_e3fd_conductivity,1
166,km,missing_range,1
172,lx,mz_shutter_ff_lara_r_illuminance,1
173,lx,plant_sensor_e3fd_illuminance,1
85,V,shellyuni_34945478fe57_adc,1
197,s,io_series_8_9_2e3c_time,1
78,Hz,sn_3015663157_metering_frequency,1
208,state,ewelink_ds01_opening,1
210,state,floorlamp2_floor_lamp_button_1,1
211,state,floorlamp2_floor_lamp_button_2,1
